In [ ]:
import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import pandas as pd


In [ ]:
start = time.time()


# At Siena
#topdir = '/mnt/qnap/babar_data/bnv_plambda'

# At Bellis' home
#topdir = 'bnv_plambda'

# On Bellis' laptop
topdir = './'

filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
#filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'

data = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")


In [ ]:
nlam = data['nLambda0']
lamfl = data['Lambda0FlightLen']
lammass= data['Lambda0_unc_Mass']
spmode= data['spmode']


d1idx = data['Lambda0d1Idx']
d2idx = data['Lambda0d2Idx']

d1lund = data['Lambda0d1Lund']
d2lund = data['Lambda0d2Lund']

Bd1idx = data['Bd1Idx']
Bd2idx = data['Bd2Idx']

Bd1lund = data['Bd1Lund']
Bd2lund = data['Bd2Lund']



plt.figure(figsize=(12,3))
plt.subplot(1,2,1)
plt.hist(nlam);

plt.subplot(1,2,2)
plt.hist(nlam[nlam!=2]);

In [ ]:
print(lamfl)
print(lammass)
print(spmode)

print()

print(d1idx)
print(d1lund)
print()

print(d2idx)
print(d2lund)
print()

print(Bd1idx)
print(Bd1lund)
print()

print(Bd2idx)
print(Bd2lund)
print()

In [ ]:
mask = lamfl >= 0

plt.figure(figsize=(12,3))
plt.subplot(1,2,1)
plt.hist(ak.flatten(lamfl),bins=100);

plt.subplot(1,2,2)
plt.hist(ak.flatten(lamfl[mask]), bins=100);

In [ ]:
mask = lamfl >= 0

plt.figure(figsize=(12,3))
plt.subplot(1,2,1)
plt.hist(ak.num(lamfl));

plt.subplot(1,2,2)
plt.hist(ak.num(lamfl[mask]));

In [ ]:
mask = nlam == 2

x1 = d1idx[mask][:,0]
x2 = d1idx[mask][:,1]

x3 = d2idx[mask][:,0]
x4 = d2idx[mask][:,1]

x5 = Bd2idx[mask][:,0]
x6 = Bd2idx[mask][:,1]

x7 = lammass[mask][:,0]
x8 = lammass[mask][:,1]


print(x1)
print(x2)
print()

print("Lambda0 d1")
print(len(x1))
print(len(x1[x1 != x2]))
print(x1[x1 != x2])
print(x2[x1 != x2])
print(spmode[x1 != x2])
print()

print("Lambda0 d2")
print(len(x3))
print(len(x3[x3 != x4]))
print(x3[x3 != x4])
print(x4[x3 != x4])
print(spmode[x3 != x4])
print()

print("B d2 (proton)")
print(len(x5))
print(len(x5[x5 != x6]))
print(x5[x5 != x6])
print(x6[x5 != x6])
print(spmode[x5 != x6])
print()

print("Lambda0 mass")
print(len(x7))
print(len(x7[x7 != x8]))
print(x7[x7 != x8])
print(x8[x7 != x8])


print(x7[x7 != x8][4])
print(x8[x7 != x8][4])

print(x7[x7 != x8][5])
print(x8[x7 != x8][5])

print(x7[x7 != x8][7])
print(x8[x7 != x8][7])


print(spmode[x7 != x8])
print()


In [ ]:
#data.fields

In [ ]:
mask = nlam == 2

x1 = d1idx[mask][:,0]
x2 = d1idx[mask][:,1]

x3 = d2idx[mask][:,0]
x4 = d2idx[mask][:,1]

x5 = Bd2idx[mask][:,0]
x6 = Bd2idx[mask][:,1]

x7 = lammass[mask][:,0]
x8 = lammass[mask][:,1]


n = 25

ppmag, pcosth, pphi = data['pp3'], data['pcosth'], data['pphi']
pipmag, picosth, piphi = data['pip3'], data['picosth'], data['piphi']

print(x1[n], x2[n], x6[n] , ppmag[n], pipmag[n], pcosth[n], picosth[n], pphi[n], piphi[n])
#ppmag

In [ ]:
mask = nlam == 2

print(d1idx[mask])

x1 = d1idx[mask][:,1]

x1